In [108]:
!pip install kagglehub

In [109]:
import kagglehub
path = kagglehub.dataset_download("fredericods/ptbr-sentiment-analysis-datasets")
print(f"Dataset salvo em: {path}")

Dataset salvo em: /home/hericson/.cache/kagglehub/datasets/fredericods/ptbr-sentiment-analysis-datasets/versions/1


![imersao02_visaoarquivo.png](./figuras/imersao02_visaoarquivo.png)

### Qual Escolha adequada??

![imersao02_objetivoescolha.png](./figuras/imersao02_objetivoescolha.png)

### Vamos trabalhar com o utcl_apps

In [110]:
## Passo 1: Importar bibliotecas
import pandas as pd
import nltk
import string
import joblib
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')


[nltk_data] Downloading package punkt to /home/hericson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hericson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/hericson/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [111]:
## Passo 2: Carregar os dados
# Certifique-se que o arquivo utcl_apps.csv esteja na mesma pasta do script
df = pd.read_csv('datasets/utlc_apps.csv')
df.head()

,original_index,review_text,review_text_processed,review_text_tokenized,polarity,rating,kfold_polarity,kfold_rating
0,2483729,eu curtindo muito,eu curtindo muito,"['eu', 'curtindo', 'muito']",1.0,5.0,1,1
1,2865805,Aplicativo absurdamente caro e o valor para se...,aplicativo absurdamente caro e o valor para se...,"['aplicativo', 'absurdamente', 'caro', 'valor'...",0.0,1.0,1,1
2,2734161,Não é mais tão simples com o novo layout,nao e mais tao simples com o novo layout,"['nao', 'mais', 'tao', 'simples', 'com', 'novo...",1.0,4.0,1,1
3,2066362,"Estava com um problema para acessar, porém con...","estava com um problema para acessar, porem con...","['estava', 'com', 'um', 'problema', 'para', 'a...",1.0,5.0,1,1
4,2521806,Eu gostei muito do jogo tenho no celular da mi...,eu gostei muito do jogo tenho no celular da mi...,"['eu', 'gostei', 'muito', 'do', 'jogo', 'tenho...",1.0,5.0,1,1


![imersao02_colunasutcl.png](./figuras/imersao02_colunasutcl.png)

In [112]:
# Conferir classes
print("Distribuição das classes:")
print(df['polarity'].value_counts())

Distribuição das classes:
polarity
1.0    750744
0.0    218114
Name: count, dtype: int64


![imersao02_escolhacolunas.png](./figuras/imersao02_escolhacolunas.png)

In [113]:
# Manter apenas colunas essenciais
df = df[['review_text', 'polarity']]

In [114]:
# Remover registros nulos (caso existam)
df = df.dropna()

In [115]:
# Remover duplicados
df = df.drop_duplicates()

In [116]:
# Função de limpeza de texto avançada
def clean_text(text):
    text = str(text).lower() ## Substituir para minúsculo
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^\w\s]', '', text)  # Remove pontuação
    text = re.sub(r'\d+', '', text)  # Remove números
    #Isso remove números que são palavras sozinhas (como "123456"), mas mantém "nota 10" ou "5 estrelas".
    text = re.sub(r'\\b\\d+\\b', '', text)


    return text

df['review_text'] = df['review_text'].apply(clean_text)

#### Remover ou não números? Depende do contexto!

➕ Argumentos a favor da remoção

- Ruído técnico: Muitos números em textos são irrelevantes, como códigos de pedido, números de protocolo, datas e IDs. Esses números geralmente não carregam sentimento. Exemplo: "Comprei dia 23/12 e recebi no dia 30" → aqui os números não agregam sentimento.

- Modelos mais simples: Remover números pode simplificar o vocabulário do modelo, reduzindo a dimensionalidade no vetor de palavras (menos termos únicos).

➖ Argumentos contra a remoção

- Números com significado emocional: Em casos como avaliações e notas (exemplo: "nota 10", "dou 5 estrelas"), o número transmite diretamente o sentimento do usuário.

- Expressões comuns: "100% recomendado", "app nota 1000" — são claras indicações de sentimento positivo.

Contexto específico: Em reviews de produtos técnicos (hardware, eletrônicos), números podem indicar características relevantes ("bateria dura 8 horas").


✅ Vamos manter os números!

Em reviews de apps, é muito comum que usuários usem números para expressar sentimentos (notas, avaliações diretas).

**Em vez de remover números, você pode tratá-los como palavras, ou até criar uma feature adicional ("contém nota numérica").**

![imersao02_numerosregras.png](./figuras/imersao02_numerosregras.png)

In [117]:
# Função de limpeza de texto avançada
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^\w\s]', '', text)  # Remove apenas caracteres especiais (mas mantém números e letras)
    return text

df['review_text'] = df['review_text'].apply(clean_text)

In [118]:
df.tail()

,review_text,polarity
1039528,banido injustamente,0.0
1039531,mt bom o app assim dá pra ficar por dentro da ...,1.0
1039532,ele é ok,0.0
1039533,excelente aplicativo para treinar os conhecime...,1.0
1039534,adoro o line,1.0


In [119]:
# Configurar stopwords e stemming
stop_words = set(stopwords.words('portuguese'))
stemmer = RSLPStemmer()

In [120]:
# Pipeline completo de pré-processamento
def preprocess_text(text):
    tokens = word_tokenize(text)
    # tokens = [stemmer.stem(word) for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)

df['review_text'] = df['review_text'].apply(preprocess_text)

In [121]:
# Salvar o dataframe tratado para análise e treino futuro
df.to_csv('datasets/processed_utcl_apps.csv', index=False)

In [122]:
df.head()

,review_text,polarity
0,eu curtindo muito,1.0
1,aplicativo absurdamente caro e o valor para se...,0.0
2,não é mais tão simples com o novo layout,1.0
3,estava com um problema para acessar porém cons...,1.0
4,eu gostei muito do jogo tenho no celular da mi...,1.0


In [123]:
print("\nResumo dos dados após o tratamento:")
print(df['polarity'].value_counts())

print("\nExemplo de dados tratados:")
print(df.head())

print("\nDataset tratado salvo como 'processed_utcl_apps.csv'")


Resumo dos dados após o tratamento:
polarity
1.0    538926
0.0    207642
Name: count, dtype: int64

Exemplo de dados tratados:
                                         review_text  polarity
0                                  eu curtindo muito       1.0
1  aplicativo absurdamente caro e o valor para se...       0.0
2           não é mais tão simples com o novo layout       1.0
3  estava com um problema para acessar porém cons...       1.0
4  eu gostei muito do jogo tenho no celular da mi...       1.0

Dataset tratado salvo como 'processed_utcl_apps.csv'


### Treinamento e Avaliação de Modelos para Análise de Sentimentos (Apps)

In [124]:
print("Tamanho total do dataset:", df.shape)
print("Amostra dos dados após limpeza:")
print(df.head())

if df.shape[0] == 0:
    raise ValueError("Erro: O dataset está completamente vazio após o pré-processamento!")

Tamanho total do dataset: (746568, 2)
Amostra dos dados após limpeza:
                                         review_text  polarity
0                                  eu curtindo muito       1.0
1  aplicativo absurdamente caro e o valor para se...       0.0
2           não é mais tão simples com o novo layout       1.0
3  estava com um problema para acessar porém cons...       1.0
4  eu gostei muito do jogo tenho no celular da mi...       1.0


In [125]:
import pandas as pd
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [126]:
# Carregar dataset pré-processado
df = pd.read_csv('datasets/processed_utcl_apps.csv')

#### *Garantir que todos os textos são strings válidas e remover entradas vazias.*

In [127]:
# Remover valores NaN ocultos
df = df.dropna(subset=['review_text'])

# Garantir que todas as entradas são strings
df['review_text'] = df['review_text'].astype(str)

# Remover espaços em branco e linhas vazias
df['review_text'] = df['review_text'].str.strip()
df = df[df['review_text'] != ""] ## Filtrar linhas que possuem texto

# Se review_text for uma lista de tokens, converter para string novamente
df['review_text'] = df['review_text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Conferir que não há mais NaN ou valores estranhos
print(df.isna().sum())  # Deve mostrar 0 para review_text
print(df[df['review_text'].apply(lambda x: not isinstance(x, str))])  # Deve estar vazio

# Agora podemos treinar sem erro


review_text    0
polarity       0
dtype: int64
Empty DataFrame
Columns: [review_text, polarity]
Index: []


In [128]:
# Separar entrada (X) e rótulo (y)
X = df['review_text']
y = df['polarity']

In [129]:
X 

0                                         eu curtindo muito
1         aplicativo absurdamente caro e o valor para se...
2                  não é mais tão simples com o novo layout
3         estava com um problema para acessar porém cons...
4         eu gostei muito do jogo tenho no celular da mi...
                                ...                        
746563                                  banido injustamente
746564    mt bom o app assim dá pra ficar por dentro da ...
746565                                             ele é ok
746566    excelente aplicativo para treinar os conhecime...
746567                                         adoro o line
Name: review_text, Length: 746232, dtype: object

In [130]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Agora que os dados estão pré-processados, escolhemos dois modelos para análise de sentimentos:

- Naive Bayes e Rede Neural MLP.

Ambos têm características distintas que ajudam a entender o impacto de técnicas diferentes na classificação de texto.

#### Modelo 1: Naive Bayes (MultinomialNB)

Por que escolhemos?

- É rápido e eficiente para classificação de texto.

- Baseia-se na probabilidade das palavras estarem associadas a cada classe (positivo, negativo).

- Funciona bem mesmo com poucos dados, já que assume independência entre palavras (embora essa seja uma simplificação forte).

In [131]:
from sklearn.preprocessing import MaxAbsScaler

In [132]:
# Modelo 1: Naive Bayes
print("\nTreinando modelo Naive Bayes...")
nb_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=100)),
    ('scaler', MaxAbsScaler()),
    ('classifier', MultinomialNB())
])

nb_pipeline.fit(X_train, y_train)

nb_predictions = nb_pipeline.predict(X_test)

print("\nRelatório de Classificação - Naive Bayes")
print(classification_report(y_test, nb_predictions))

# Salvar modelo
joblib.dump(nb_pipeline, 'nb_model_apps.pkl')


Treinando modelo Naive Bayes...

Relatório de Classificação - Naive Bayes
              precision    recall  f1-score   support

         0.0       0.89      0.05      0.10     62398
         1.0       0.73      1.00      0.84    161472

    accuracy                           0.73    223870
   macro avg       0.81      0.52      0.47    223870
weighted avg       0.78      0.73      0.64    223870



['nb_model_apps.pkl']

In [133]:
nb_pipeline.named_steps['vectorizer'].get_feature_names_out()

array(['agora', 'ainda', 'amei', 'ao', 'aplicativo', 'app', 'as',
       'atualização', 'até', 'baixar', 'bem', 'bom', 'celular', 'com',
       'como', 'consigo', 'conta', 'da', 'de', 'depois', 'dia', 'do',
       'dos', 'ele', 'em', 'erro', 'esse', 'esta', 'estou', 'estrelas',
       'está', 'eu', 'excelente', 'favor', 'fazer', 'fica', 'foi',
       'funciona', 'fácil', 'gostei', 'gosto', 'hora', 'isso', 'ja',
       'jogo', 'já', 'legal', 'mais', 'mas', 'me', 'melhor', 'mesmo',
       'meu', 'minha', 'muito', 'na', 'nada', 'nao', 'nem', 'no', 'não',
       'os', 'ou', 'para', 'parabéns', 'pelo', 'pois', 'por', 'porque',
       'pq', 'pra', 'problema', 'quando', 'que', 'quem', 'recomendo',
       'ruim', 'se', 'sem', 'sempre', 'ser', 'so', 'super', 'só', 'ta',
       'tem', 'tempo', 'tenho', 'ter', 'todos', 'trava', 'tudo', 'um',
       'uma', 'usar', 'vc', 'ver', 'vezes', 'vou', 'ótimo'], dtype=object)

1) Explicação das Métricas

**Precision (Precisão): Mede quantas das predições positivas estão corretas.**

*Interpretação dos valores:

0.79 para a classe 0.0 (Negativo) significa que 79% das avaliações que o modelo classificou como "Negativo" realmente são negativas.
0.91 para a classe 1.0 (Positivo) significa que 91% das avaliações que o modelo classificou como "Positivo" realmente são positivas.

O modelo tem maior precisão para identificar avaliações positivas, mas ainda tem uma boa precisão para avaliações negativas.

**Recall (Revocação ou Sensibilidade): Mede quantos dos exemplos reais de uma classe foram corretamente classificados.**

*Interpretação dos valores:

0.78 para a classe 0.0 (Negativo) → O modelo consegue capturar 78% dos comentários realmente negativos.
0.92 para a classe 1.0 (Positivo) → O modelo consegue capturar 92% dos comentários realmente positivos.

O modelo está mais eficiente para identificar avaliações positivas do que negativas, pois captura mais exemplos corretos da classe positiva.

**F1-score (Média Harmônica de Precision e Recall): É uma média ponderada de Precision e Recall, útil para balancear modelos.**

0.78 para avaliações negativas e 0.92 para positivas significa que o modelo tem um desempenho melhor na classificação de avaliações positivas.

A F1-score de 0.92 para positivos indica um modelo bem treinado para essa classe, mas a F1-score de 0.78 para negativos sugere que ele pode estar confundindo algumas avaliações negativas.

**Support (Suporte): Mostra quantas amostras reais existem para cada classe.**

- 62.035 exemplos negativos.
- 161.786 exemplos positivos.

Há mais do que o dobro de avaliações positivas do que negativas. Isso pode estar influenciando o modelo a ser mais preciso na classe positiva.

2) Interpretação dos Agregados (Médias)

**Accuracy (Acurácia): Mede quantas predições o modelo acertou no total.**

88% de acurácia → O modelo está certo em 88% das avaliações. O modelo tem um ótimo desempenho geral!

**Macro Avg (Média Macro): Tira a média simples entre as classes:**

Aqui, a macro média é 0.85, o que significa que o modelo tem um bom equilíbrio, mas pode melhorar para a classe negativa.

**Weighted Avg (Média Ponderada): Leva em consideração o suporte (quantidade de exemplos por classe).**

Como há mais exemplos positivos, esse valor fica mais próximo dos resultados da classe positiva.

A Weighted Avg de 0.88 reflete o desempenho real do modelo, já que a classe positiva tem muito mais exemplos.

**Resumo Final:**

1) O modelo funciona muito bem com 88% de acurácia geral.

2) Ele é muito bom para identificar avaliações positivas (F1-score de 0.92).

3) Ele tem um pouco mais de dificuldade com avaliações negativas (F1-score de 0.78).

4) Isso pode ser causado pelo desbalanceamento dos dados (muito mais positivos do que negativos).


**O que podemos melhorar?**

- Equilibrar as classes: Podemos tentar técnicas como undersampling (remover positivos) ou oversampling (aumentar negativos) para balancear melhor o treino.

- Testar outro modelo: Naive Bayes é rápido.

---

#### Modelo 2: Rede Neural MLP (Multilayer Perceptron)


Por que escolhemos?

- Pode capturar padrões mais complexos do que Naive Bayes.

- Melhor adaptação a dados ruidosos, pois aprende representações internas dos textos.

- Treinamento mais demorado, mas pode ter maior precisão se bem ajustado.

In [ ]:
# Modelo 2: Rede Neural MLP
print("\nTreinando modelo MLP (Rede Neural)...")
mlp_pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),  # Vetorização do texto
    ('classifier', MLPClassifier(max_iter=50, hidden_layer_sizes=(50,)))  # MLP com uma camada oculta de 50 neurônios
])

mlp_pipeline.fit(X_train, y_train)  # Treina o modelo
mlp_predictions = mlp_pipeline.predict(X_test)  # Faz predições

print("\nRelatório de Classificação - MLP")
print(classification_report(y_test, mlp_predictions))  # Exibe métricas

# Salvar modelo
joblib.dump(mlp_pipeline, 'mlp_model_apps.pkl')



Treinando modelo MLP (Rede Neural)...


#### Comparação entre os dois modelos

![imersao2-comparacaomodelos.png](./figuras/imersao2-comparacaomodelos.png)

- Se o seu objetivo for rapidez e facilidade de uso, Naive Bayes pode ser melhor.

- Se quiser um modelo mais robusto e flexível, a Rede Neural MLP pode dar melhores resultados.

### Testando os dois modelos

In [ ]:
# Código para realizar predição a partir do teclado
def fazer_predicao(modelo):
    print("\nDigite um comentário para análise de sentimento:")
    comentario = input()
    sentimento = modelo.predict([comentario])[0]  # Faz a predição do sentimento
    print(f"\nO sentimento previsto é: {sentimento}")

In [ ]:
# Carregar e testar predição MODELO NB
print("\nTestando predição com Naive Bayes")
modelo_nb = joblib.load('nb_model_apps.pkl')  # Carrega modelo salvo
fazer_predicao(modelo_nb)

In [ ]:
# Carregar e testar predição MODELO MLP
print("\nTestando predição com MLP")
modelo_mlp = joblib.load('mlp_model_apps.pkl')  # Carrega modelo salvo
fazer_predicao(modelo_mlp)